In [1]:
import sys
sys.path.insert(0, '..')
import numpy as np
from BinaryTokenizer import BinaryTokenizer
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn as nn

In [2]:
npz_path = '../data/augmented_and_padded_data.npz'

In [3]:
class TokenizedChromaDataset(Dataset):
    def __init__(self, npz_path):
        data = np.load(npz_path)
        melody_pcps = data['melody_pcps']
        chord_pcps = data['chord_pcps']
        binTok = BinaryTokenizer()
        self.tok_melody = binTok.fit_transform( melody_pcps )
        self.tok_chord = binTok.fit_transform( chord_pcps )
    # end __init__
    
    def __len__(self):
        return self.tok_melody.shape[0]
    # end __len__
    
    def __getitem__(self, idx):
        return self.tok_melody[idx,:], self.tok_chord[idx,:]
    # end __getitem__
# end BinChromaDataset

In [4]:
class BinChromaDataset(Dataset):
    def __init__(self, npz_path):
        data = np.load(npz_path)
        self.melody_pcps = data['melody_pcps'].astype('float32')
        self.chord_pcps = data['chord_pcps'].astype('float32')
    # end __init__
    
    def __len__(self):
        return self.melody_pcps.shape[0]
    # end __len__
    
    def __getitem__(self, idx):
        return self.melody_pcps[idx,:,:], self.chord_pcps[idx,:,:]
    # end __getitem__
# end BinChromaDataset

In [5]:
td = TokenizedChromaDataset(npz_path)
td_loader = DataLoader(td, batch_size=64, shuffle=True)
bd = BinChromaDataset(npz_path)
bd_loader = DataLoader(bd, batch_size=64, shuffle=True)
# split_idx = int( len(td)*0.8 )
# train_set = Subset(td, range(0,split_idx))
# test_set = Subset(td, range(split_idx, len(dataset)))

In [13]:
# train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
# test_loader = DataLoader(test_set, batch_size=64, shuffle=True)

In [10]:
# melodies, chords = next(iter(train_loader))
t_melodies, t_chords = next(iter(td_loader))
print(t_melodies.shape)
print(t_chords.shape)

torch.Size([64, 129])
torch.Size([64, 129])


In [11]:
b_melodies, b_chords = next(iter(bd_loader))
print(b_melodies.shape)
print(b_chords.shape)

torch.Size([64, 129, 12])
torch.Size([64, 129, 12])


In [12]:
embedding = nn.Embedding(2**12, 512)
linear = nn.Linear(12,512)

In [14]:
y_embedding = embedding(t_melodies)
print(y_embedding.shape)

torch.Size([64, 129, 512])


In [16]:
y_linear = linear(b_melodies)
print(y_linear.shape)

torch.Size([64, 129, 512])
